## 3 - Extract Prices
> Get price change for all stocks on document's release date

### 3.0 - Import Libraries

In [2]:
import pandas as pd
import numpy as np
import time
import random
import unicodedata
import re
import datetime
import os
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
import ast
import requests
from bs4 import BeautifulSoup
import pandas_datareader.data as web
import pandas_market_calendars as mcal
from tqdm import tqdm

In [3]:
# Set max columns and width with pandas
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

In [5]:
# Clear future and deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

### 3.1 Read in Data

In [13]:
df = pd.read_csv('../clean_data/concat.csv')
df.drop(columns=['index'], axis=1, inplace=True)

### 3.2 Drop Duplicates

In [14]:
# Check shape before
df.shape

(63659, 11)

In [18]:
# Override DataFrame dropping duplicate values
df = df.drop_duplicates(subset=['corpus', 'release_date'])

In [19]:
# Check shape after
df.shape

(57681, 11)

In [20]:
# Check DataFrame's first row
df.head(1)

,company,date,doc_link,filing_doc,complete_file_link,file_description,security,gics_sector,gics_sub_industry,release_date,corpus
0,CSCO,2019-08-22,http://sec.gov/Archives/edgar/data/858877/0001...,8-K,https://sec.gov/Archives/edgar/data/858877/000...,Complete submission text file,Cisco Systems,Information Technology,Communications Equipment,2019-08-22 16:35:32,|Form 8-K UNITED STATES SECURITIES AND EXCHANG...


#### Drop Columns `doc_link`, `complete_file_link`, `file_description`, `filing_doc`, `date`

In [21]:
df.drop(columns=['doc_link', 'complete_file_link', 'file_description', 'filing_doc', 'date'], inplace=True)
df.head(1)

,company,security,gics_sector,gics_sub_industry,release_date,corpus
0,CSCO,Cisco Systems,Information Technology,Communications Equipment,2019-08-22 16:35:32,|Form 8-K UNITED STATES SECURITIES AND EXCHANG...


#### Check for Null Values and Drop Them

In [22]:
df.isnull().sum()

company              0
security             0
gics_sector          0
gics_sub_industry    0
release_date         1
corpus               1
dtype: int64

In [23]:
df.dropna(inplace=True)

### 3.3 Create `tokens` column - Tokenize `corpus`

In [24]:
# Create function to tokenize 'body'
def tokenizer(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)

In [25]:
df['tokens'] = df['corpus'].progress_map(tokenizer)

### 3.4 Create `length_corpus` column

In [26]:
# Create column with corpus lenght
df['length_corpus'] = df['corpus'].progress_map(lambda x: len(x))

### 3.5 Create `items` Columns
>Learn More: [8-K Filings Item Types](www.sec.gov/fast-answers/answersform8khtm.html)

In [27]:
# Define function to extract document types
def extract_item(text):
    '''Extracts Types of Documents in each Filing'''
    # Compile Regular Expression
    exp = re.compile("Item+ +\d+[\:,\.]+\d+\d")
    
    # Create list with all items found
    item_list = re.findall(exp, text)
    item_list = [i.replace(' ', '').strip() for i in item_list]
    
    # Return list
    return item_list

In [28]:
# Create column with items types in each document
df['items'] = df['corpus'].progress_map(extract_item)

#### Convert `release_date` to Datetime

In [29]:
df['release_date'] = pd.to_datetime(df['release_date'])

### 3.6 - Get Stock Prices from Yahoo Finance
> _Yahoo! Finance is a media property that is part of Yahoo!'s network. It provides financial news, data and commentary including stock quotes, press releases, financial reports, and original content. It also offers some online tools for personal finance management._ [Learn More](https://finance.yahoo.com/)

> _Remote Data Access with Pandas DataReader_ [Documentation](https://pandas-datareader.readthedocs.io/en/latest/)

#### Define Function to Get Prices

In [30]:
def get_prices(ticker_list, start_date, end_date):
    '''
    Get Stock Prices from Yahoo Finance for Specific Time Frame
    Inputs = List of Tickers, Start Date and End Date
    Output = DataFrame
    '''
    # Instantiate DataFrame to house prices
    prices_df = pd.DataFrame()

    # Loop through all companies in the data
    for ticker in tqdm(ticker_list):

        # Get prices
        ticker_df = web.DataReader(ticker ,'yahoo', start_date, end_date)

        # Create columns with company's ticker
        ticker_df['company'] = ticker

        # Concatenate DataFrame with prices_df
        prices_df = pd.concat([prices_df, ticker_df], axis=0)
    
    # Remove data from index
    prices_df.reset_index(inplace=True)
    
    # Lower columns names
    prices_df.columns = [column.lower() for column in prices_df.columns]

    # Return DataFrame with all companies prices for requested timeframe
    return prices_df

In [31]:
prices_df = get_prices(df['company'].unique(), '2010-01-01', '2019-12-31')

In [32]:
prices_df.head(1)

,date,high,low,open,close,volume,adj close,company
0,2010-01-04,24.84,24.01,24.110001,24.690001,59853700.0,19.253393,CSCO


#### Define Functions to Calculate Price Change

In [33]:
# Get list of holidays for the New York Stock Exchange
holidays = mcal.get_calendar('NYSE').holidays().holidays

def date_check(release_date):
    '''Checks for Holidays and Weekends'''
    while release_date.isoweekday() > 5 or release_date.date() in holidays:
        release_date = release_date + datetime.timedelta(days=-1)
    return release_date

In [34]:
def get_percentage_change(ticker, release_date, dataframe):    
    '''Calculates Price Change for a Security Given Document Release Date'''
    # define market hours
    market_open = release_date.replace(hour=9,minute=30,second=0)
    market_close = release_date.replace(hour=16,minute=0,second=0)
    
    try:
        # release_date BEFORE market open
        if release_date < market_open:

            # Get open price
            open_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == release_date \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['open'].item()

            # Get close price
            close_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

            percentage_change = (open_price - close_price) / close_price

        # release_date AFTER market close
        elif release_date > market_close:
            
            # Get open price
            open_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == date_check(release_date + datetime.timedelta(days=+1))\
                                                .strftime('%m/%d/%Y').replace('/','-'))]['open'].item()

            # Get close price
            close_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == release_date \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

            # Percentage change if release_date > market_close
            percentage_change = (open_price - close_price) / close_price

        # release_date during trading hours
        else:
            # Get open price
            open_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == release_date \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['open'].item()

            # Get close price
            close_price = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == release_date \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

            percentage_change = (close_price - open_price) / open_price
            
    except:
        percentage_change = np.nan
        
    return percentage_change

#### Create Column with Percentage Change ( Stocks )
> _Explain Step_

In [36]:
df['stock_change'] = [get_percentage_change(df['company'][i],
                                            df['release_date'][i],
                                            prices_df) for i in tqdm(df.index)]

#### Create Column with Percentage Change for The S&P 500 Index ( GSPC )
> _The S&P 500, or just the S&P, is a stock market index that measures the stock performance of 500 large companies listed on stock exchanges in the United States. It is one of the most commonly followed equity indices, and many consider it to be one of the best representations of the U.S. stock market.The average annual total return of the index, including dividends, since inception in 1926 has been 9.8%; however, there were several years where the index declined over 30%.The index has posted annual increases 70% of the time._ [Learn More](https://en.wikipedia.org/wiki/S%26P_500_Index)

In [37]:
sp500_vix_df = get_prices(['^GSPC', '^VIX'], '2010-01-01', '2019-12-31')

In [38]:
sp500_vix_df.head(1)

,date,high,low,open,close,volume,adj close,company
0,2010-01-04,1133.869995,1116.560059,1116.560059,1132.98999,3991400000,1132.98999,^GSPC


In [39]:
df['index_change'] = [get_percentage_change('^GSPC',
                                        df['release_date'][i],
                                        sp500_vix_df) for i in tqdm(df.index)]

#### Create Column with Normalized Percentage Change
> _Explain Step_

In [40]:
df['normalized_change'] = df['stock_change'] - df['index_change']

#### Create Column with Percentage Change for The CBOE Volatility Index ( VIX )
> _The CBOE Volatility Index, known by its ticker symbol VIX, is a popular measure of the stock market's expectation of volatility implied by S&P 500 index options. It is calculated and disseminated on a real-time basis by the Chicago Board Options Exchange (CBOE), and is commonly referred to as the fear index or the fear gauge._ [Learn More](https://en.wikipedia.org/wiki/VIX)

In [41]:
def get_vix(release_date, dataframe):
    
    # define market hours
    market_open = release_date.replace(hour=9,minute=30,second=0)
    market_close = release_date.replace(hour=16,minute=0,second=0)
    
    try:
        if release_date < market_open:
            vix_price = dataframe.loc[(dataframe['company'] == '^VIX') & \
                                  (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()


    
        elif release_date > market_open:
            vix_price = dataframe.loc[(dataframe['company'] == '^VIX') & \
                                  (dataframe['date'] == release_date \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
        
        else:
            vix_price = dataframe.loc[(dataframe['company'] == '^VIX') & \
                                  (dataframe['date'] == release_date \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['open'].item()
    
    except:
        vix_price = np.nan
        
    return vix_price

In [42]:
df['vix'] = [get_vix(df['release_date'][i], sp500_vix_df) for i in tqdm(df.index)]

In [43]:
df.head(1)

,company,security,gics_sector,gics_sub_industry,release_date,corpus,tokens,length_corpus,items,stock_change,index_change,normalized_change,vix
0,CSCO,Cisco Systems,Information Technology,Communications Equipment,2019-08-22 16:35:32,|Form 8-K UNITED STATES SECURITIES AND EXCHANG...,"[Form, 8, K, UNITED, STATES, SECURITIES, AND, ...",3233,[Item8.01],-0.005812,-0.004064,-0.001747,16.68


#### Get Historical Movements

In [46]:
def get_historical(ticker, release_date, period, dataframe=prices_df):
        
    try:
        # Percentage change for the last week        
        if period == 'w':
            start = dataframe.loc[(dataframe['company'] == ticker) & \
                                  (dataframe['date'] == date_check(release_date + datetime.timedelta(weeks=-1)) \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
            end = dataframe.loc[(dataframe['company'] == ticker) & \
                          (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

        # Percentage change for the last month
        elif period == 'm':
            start = dataframe.loc[(dataframe['company'] == ticker) & \
                                  (dataframe['date'] == date_check(release_date + datetime.timedelta(weeks=-4)) \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
            end = dataframe.loc[(dataframe['company'] == ticker) & \
                                (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                      .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

        # Percentage change for the last quarter
        elif period == 'q':
            start = dataframe.loc[(dataframe['company'] == ticker) & \
                                  (dataframe['date'] == date_check(release_date + datetime.timedelta(weeks=-12)) \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
            end = dataframe.loc[(dataframe['company'] == ticker) & \
                                (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                      .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()

        # Percentage change for the last year
        elif period == 'y':
            start = dataframe.loc[(dataframe['company'] == ticker) & \
                                  (dataframe['date'] == date_check(release_date + datetime.timedelta(weeks=-52)) \
                                                        .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
            end = dataframe.loc[(dataframe['company'] == ticker) & \
                                (dataframe['date'] == date_check(release_date + datetime.timedelta(days=-1)) \
                                                      .strftime('%m/%d/%Y').replace('/','-'))]['close'].item()
    except:
        return np.nan
            
    return (end - start) / start

#### Week - 1

In [47]:
df['hist_week'] = [get_historical(df['company'][i],
                                  df['release_date'][i],
                                  period='w',
                                  dataframe=prices_df) for i in tqdm(df.index)]

#### Month - 1

In [48]:
df['hist_month'] = [get_historical(df['company'][i],
                                  df['release_date'][i],
                                  period='m',
                                  dataframe=prices_df) for i in tqdm(df.index)]

#### Quarter - 1

In [49]:
df['hist_quarter'] = [get_historical(df['company'][i],
                                  df['release_date'][i],
                                  period='q',
                                  dataframe=prices_df) for i in tqdm(df.index)]

#### Year - 1

In [50]:
df['hist_year'] = [get_historical(df['company'][i],
                                  df['release_date'][i],
                                  period='y',
                                  dataframe=prices_df) for i in tqdm(df.index)]

In [51]:
df.head(1)

,company,security,gics_sector,gics_sub_industry,release_date,corpus,tokens,length_corpus,items,stock_change,index_change,normalized_change,vix,hist_week,hist_month,hist_quarter,hist_year
0,CSCO,Cisco Systems,Information Technology,Communications Equipment,2019-08-22 16:35:32,|Form 8-K UNITED STATES SECURITIES AND EXCHANG...,"[Form, 8, K, UNITED, STATES, SECURITIES, AND, ...",3233,[Item8.01],-0.005812,-0.004064,-0.001747,16.68,0.054486,-0.138644,-0.089602,0.059757


### 3.7 - Drop Rows with Errors

In [52]:
df.shape

(57680, 17)

In [53]:
df.isnull().sum()

company                0
security               0
gics_sector            0
gics_sub_industry      0
release_date           0
corpus                 0
tokens                 0
length_corpus          0
items                  0
stock_change         271
index_change         127
normalized_change    271
vix                  111
hist_week            165
hist_month           207
hist_quarter         282
hist_year            637
dtype: int64

In [55]:
df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [56]:
df.shape

(56911, 17)

### 3.8 Create `movement` Column
> Based on the normalized change column classify observations as being greater than 1% ( Up ), less than -1% ( Down ) or in between -1% and 1% ( Neutral ).

In [59]:
df['movement'] = df['normalized_change'].map(lambda x: 'down' if x < -.01 else ('up' if x > .01 else 'neutral'))

In [60]:
df.head(1)

,company,security,gics_sector,gics_sub_industry,release_date,corpus,tokens,length_corpus,items,stock_change,index_change,normalized_change,vix,hist_week,hist_month,hist_quarter,hist_year,movement
0,CSCO,Cisco Systems,Information Technology,Communications Equipment,2019-08-22 16:35:32,|Form 8-K UNITED STATES SECURITIES AND EXCHANG...,"[Form, 8, K, UNITED, STATES, SECURITIES, AND, ...",3233,[Item8.01],-0.005812,-0.004064,-0.001747,16.68,0.054486,-0.138644,-0.089602,0.059757,neutral


#### Save DataFrame

In [61]:
df.to_csv('../clean_data/corpus.csv', index_label=False)